In [2]:
import pandas as pd
import numpy as np
from functools import reduce

# Clean Merged ArcGIS Data

In [3]:
# Reading in data
df = pd.read_excel("Processed Data/ArcGIS_SpaitialJoin_TableToExcel.xlsx")

In [4]:
# Here are the first five lines
df.head()

,OBJECTID,Join_Count,TARGET_FID,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP_CODE,LATITUDE,LONGITUDE,LOCATION,...,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,CT2020_GEOID,Shape_Leng
0,1,1,1,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"(40.687534, -73.9775)",...,Brooklyn,3300.0,3003300.0,,Fort Greene,BK0203,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,3.604700e+10,6350.513735
1,2,1,2,2021-12-14,16:25,NaN,NaN,40.784615,-73.953964,"(40.784615, -73.953964)",...,Manhattan,15801.0,1015801.0,,Upper East Side-Carnegie Hill,MN0802,MN08,MN08 Upper East Side-Roosevelt Island (CD 8 Eq...,3.606102e+10,4692.931885
2,3,1,3,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"(40.59207, -73.96299)",...,Brooklyn,39000.0,3039000.0,,Gravesend (East)-Homecrest,BK1501,BK15,BK15 Sheepshead Bay-Gravesend (East) (CD 15 Ap...,3.604704e+10,6671.629240
3,4,1,4,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"(40.640835, -73.98967)",...,Brooklyn,22400.0,3022400.0,,Borough Park,BK1202,BK12,BK12 Borough Park-Kensington (CD 12 Approximat...,3.604702e+10,5835.731956
4,5,1,5,2021-12-08,19:30,MANHATTAN,10022.0,40.761750,-73.968990,"(40.76175, -73.96899)",...,Manhattan,11203.0,1011203.0,,East Midtown-Turtle Bay,MN0604,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,3.606101e+10,3682.036283


In [5]:
# What is the shape of the data? # rows and # cols
df.shape

(23455, 44)

In [6]:
# Here are all the column names
df.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'CRASH_DATE', 'CRASH_TIME',
       'BOROUGH', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'ON_STREET_NAME', 'CROSS_STREET_NAME', 'OFF_STREET_NAME',
       'NUMBER_OF_PERSONS_INJURED', 'NUMBER_OF_PERSONS_KILLED',
       'NUMBER_OF_PEDESTRIANS_INJURED', 'NUMBER_OF_PEDESTRIANS_KILLED',
       'NUMBER_OF_CYCLIST_INJURED', 'NUMBER_OF_CYCLIST_KILLED',
       'NUMBER_OF_MOTORIST_INJURED', 'NUMBER_OF_MOTORIST_KILLED',
       'CONTRIBUTING_FACTOR_VEHICLE_1', 'CONTRIBUTING_FACTOR_VEHICLE_2',
       'CONTRIBUTING_FACTOR_VEHICLE_3', 'CONTRIBUTING_FACTOR_VEHICLE_4',
       'CONTRIBUTING_FACTOR_VEHICLE_5', 'COLLISION_ID', 'VEHICLE_TYPE_CODE_1',
       'VEHICLE_TYPE_CODE_2', 'VEHICLE_TYPE_CODE_3', 'VEHICLE_TYPE_CODE_4',
       'VEHICLE_TYPE_CODE_5', 'CTLabel', 'BoroCode', 'BoroName', 'CT2020',
       'BoroCT2020', 'CDEligibil', 'NTAName', 'NTA2020', 'CDTA2020',
       'CDTANAME', 'CT2020_GEOID', 'Shape_Leng'],
      dtype='object')

In [7]:
# Create a copy of the dataframe
df1 = df.copy()

In [8]:
# Convert the GEOID column from float64 (e.g. 3.604700e+10) to int64 (36005000100)
df1['CT2020_GEOID'] = df1['CT2020_GEOID'].astype('Int64')
df1['CT2020_GEOID']

0        36047003300
1        36061015801
2        36047039000
3        36047022400
4        36061011203
            ...     
23450    36061002902
23451    36047090000
23452    36047043100
23453    36047047600
23454    36061007600
Name: CT2020_GEOID, Length: 23455, dtype: Int64

In [9]:
# Checking -- note: there will still be dozens of records that don't have GEOID, those crashes usually are on bridges or turnpike
df1[df1['CT2020_GEOID'].isnull()][['CRASH_DATE','CRASH_TIME','BOROUGH','ZIP_CODE','LATITUDE','LONGITUDE', 'ON_STREET_NAME','CROSS_STREET_NAME']]

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP_CODE,LATITUDE,LONGITUDE,ON_STREET_NAME,CROSS_STREET_NAME
5848,2020-09-15,9:25,QUEENS,11426.0,40.726040,-73.718290,JAMAICA AVENUE,COMMONWEALTH BOULEVARD
15752,2018-08-29,18:00,NaN,NaN,40.704422,-73.994910,BROOKLYN BRIDGE,NaN
16486,2018-08-13,0:10,NaN,NaN,40.714436,-73.974754,WILLIAMSBURG BRIDGE INNER ROADWA,NaN
16490,2018-08-26,7:00,NaN,NaN,40.704388,-73.994576,BROOKLYN BRIDGE,NaN
16605,2018-07-10,7:20,NaN,NaN,40.758340,-73.957750,QUEENSBORO BRIDGE LOWER ROADWAY,NaN
17333,2018-07-03,7:58,NaN,NaN,40.704388,-73.994576,BROOKLYN BRIDGE,NaN
17864,2018-05-01,18:10,NaN,NaN,40.714314,-73.974810,WILLIAMSBURG BRIDGE INNER ROADWA,NaN
17941,2018-04-24,8:30,NaN,NaN,40.758340,-73.957750,QUEENSBORO BRIDGE LOWER ROADWAY,NaN
19023,2017-12-21,17:15,NaN,NaN,40.704422,-73.994910,BROOKLYN BRIDGE,NaN
20054,2017-10-10,14:40,NaN,NaN,40.709305,-73.991670,MANHATTAN BR UPPER,NaN


In [10]:
# Drop the columns created because of ArcGIS's "spatial join" operation (6 join * 3 col/join = 18 columns)
ArcGIS_drop_columns=['OBJECTID', 'Join_Count', 'TARGET_FID']

# Drop all columns in census tract dataset except 'CT2020_GEOID'
census_tract_drop_columns = [
    # 'CT2020_GEOID'
    'CTLabel', 
    'BoroCode', 
    'BoroName',
    'CT2020', 
    'BoroCT2020', 
    'CDEligibil', 
    'NTAName', 
    'NTA2020', 
    'CDTA2020',
    'CDTANAME', 
    'Shape_Leng'
]

# Drop unuseful columns from crash datasets
crash_drop_columns = [
	# 'CRASH_DATE',
    'CRASH_TIME',
    'BOROUGH',
    # 'ZIP_CODE',
    'LATITUDE',
    'LONGITUDE',
    'LOCATION',
    'ON_STREET_NAME',
    'CROSS_STREET_NAME',
    'OFF_STREET_NAME',
    # 'NUMBER_OF_PERSONS_INJURED',
    # 'NUMBER_OF_PERSONS_KILLED',
    'NUMBER_OF_PEDESTRIANS_INJURED',
    'NUMBER_OF_PEDESTRIANS_KILLED',
    # 'NUMBER_OF_CYCLIST_INJURED',
    # 'NUMBER_OF_CYCLIST_KILLED',
    'NUMBER_OF_MOTORIST_INJURED',
    'NUMBER_OF_MOTORIST_KILLED',
    'CONTRIBUTING_FACTOR_VEHICLE_1',
    'CONTRIBUTING_FACTOR_VEHICLE_2',
    'CONTRIBUTING_FACTOR_VEHICLE_3',
    'CONTRIBUTING_FACTOR_VEHICLE_4',
    'CONTRIBUTING_FACTOR_VEHICLE_5',
    'COLLISION_ID',
    'VEHICLE_TYPE_CODE_1',
    'VEHICLE_TYPE_CODE_2',
    'VEHICLE_TYPE_CODE_3',
    'VEHICLE_TYPE_CODE_4',
    'VEHICLE_TYPE_CODE_5',
]

df2 = df1.drop(columns=ArcGIS_drop_columns+census_tract_drop_columns+crash_drop_columns)
df1.shape[1] - df2.shape[1]

37

In [11]:
# Here are the first five lines
df2.head()

,CRASH_DATE,ZIP_CODE,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,NUMBER_OF_CYCLIST_INJURED,NUMBER_OF_CYCLIST_KILLED,CT2020_GEOID
0,2021-12-14,11217.0,1.0,0.0,1,0,36047003300
1,2021-12-14,NaN,1.0,0.0,1,0,36061015801
2,2021-12-09,11223.0,1.0,0.0,1,0,36047039000
3,2021-12-09,11218.0,1.0,0.0,1,0,36047022400
4,2021-12-08,10022.0,1.0,0.0,1,0,36061011203


### Create time-related variables from CRASH DATE

In [12]:
df3 = df2.copy()

In [22]:
# Create a numerical variable column to indicate the year of crash accident
df3['CRASH_YEAR'] = pd.to_datetime(df3['CRASH_DATE']).dt.year

In [ ]:
# # Create a numerical variable column to indicate the month of crash accident
# df3['CRASH_MONTH'] = pd.to_datetime(df3['CRASH_DATE']).dt.month

In [13]:
# Create a numerical variable column to indicate the year+month of crash accident
df3['CRASH_YEAR-MONTH'] = pd.to_datetime(df3['CRASH_DATE']).dt.to_period('m')
df3['CRASH_YEAR-MONTH']

0        2021-12
1        2021-12
2        2021-12
3        2021-12
4        2021-12
          ...   
23450    2017-01
23451    2017-01
23452    2017-01
23453    2017-01
23454    2017-01
Name: CRASH_YEAR-MONTH, Length: 23455, dtype: period[M]

# Merge All Census Data to the Merged ArcGIS Data

### Merge all census data

In [14]:
# Import census data
df_acs2017 = pd.read_csv("Raw Data/Raw Data in txt File for American Community Survey (ACS) 5-Year Estimates/ACS_2013-2017.txt",sep='\t')
df_acs2018 = pd.read_csv("Raw Data/Raw Data in txt File for American Community Survey (ACS) 5-Year Estimates/ACS_2014-2018.txt",sep='\t')
df_acs2019 = pd.read_csv("Raw Data/Raw Data in txt File for American Community Survey (ACS) 5-Year Estimates/ACS_2015-2019.txt",sep='\t')
df_acs2020 = pd.read_csv("Raw Data/Raw Data in txt File for American Community Survey (ACS) 5-Year Estimates/ACS_2016-2020.txt",sep='\t')
df_acs2021 = pd.read_csv("Raw Data/Raw Data in txt File for American Community Survey (ACS) 5-Year Estimates/ACS_2017-2021.txt",sep='\t')

In [26]:
def process_acs_data(dataframe, year):
    '''
    This function takes in one single ACS dataframe and its corresponding year, cleans it, and outputs the dataframe.
    '''
    df = dataframe.copy()

    # Convert the data type for later join operation & create as a new column under the same name as census tract data
    # Census tract uses CT2020_GEOID, ACS uses Geo_FIPS
    df['CT2020_GEOID'] = df['Geo_FIPS'].astype('Int64')

    # Keep the borough column by renaming it before dropping all the others
    df['borough'] = df['Geo_COUNTY']

    # Drop all columns that start with "Geo_" (ie. geo data, non-demographic data)
    df = df.loc[:,~df.columns.str.startswith('Geo_')]

    # Rename columns
    # A00001_001:     Total Population
    # A00002_002:     Population Density (Per Sq. Mile)
    # B12001_001:     Population 25 Years and Over
    # B12001_002:     Population 25 Years and Over: Less than High School
    # B12001_003:     Population 25 Years and Over: High School Diploma
    # B12001_004:     Population 25 Years and Over: Bachelor's Degree or Better
    # A14006_001:     Median Household Income (In 2021 Inflation Adjusted Dollars) [Dollars adjusted for inflation to match value in 2021]
    # A09005_001:     Workers 16 Years and Over:
    # A09005_002:     Workers 16 Years and Over: Car, Truck, or Van
    # A09005_003:     Workers 16 Years and Over: Public Transportation (Includes Taxicab)
    # A09005_005:     Workers 16 Years and Over: Bicycle
    # A09003_001:     Average Commute to Work (In Min)
    df = df.rename({
        'SE_A00001_001':'ttl_pop',
        'SE_A00002_002':'pop_density_per_sq_mil',
        'SE_B12001_001':'pop_25_yr_over',
        'SE_B12001_002':'educ_less_hs',
        'SE_B12001_003':'educ_hs',
        'SE_B12001_004':'educ_bs_over',
        'SE_A14006_001':'median_household_inc',
        'SE_A09005_001':'workers_16_yr_over',
        'SE_A09005_002':'tranport_mean_car',
        'SE_A09005_003':'tranport_mean_public',
        'SE_A09005_005':'tranport_mean_bike',
        'SE_A09003_001':'avg_commmute_to_work_min'
        }, axis='columns')

    # Drop all remaining columns that start with "SE"
    df = df.loc[:,~df.columns.str.startswith('SE_')]

    # Compute the "population over 25 years and over for education"
    df['educ_less_hs_pct'] = df['educ_less_hs']/df['pop_25_yr_over']
    df['educ_hs_pct'] = df['educ_hs']/df['pop_25_yr_over']
    df['educ_bs_over_pct'] = df['educ_bs_over']/df['pop_25_yr_over']

    # Compute the "workers over 16 years and over for tranportation mean"
    df['tranport_mean_car_pct'] = df['tranport_mean_car']/df['workers_16_yr_over']
    df['tranport_mean_public_pct'] = df['tranport_mean_public']/df['workers_16_yr_over']
    df['tranport_mean_bike_pct'] = df['tranport_mean_bike']/df['workers_16_yr_over']

    # Drop the columns after we finished the computation
    drop_columns = [
        'pop_25_yr_over',
        'educ_less_hs',
        'educ_hs',
        'educ_bs_over',
        'workers_16_yr_over',
        'tranport_mean_car',
        'tranport_mean_public',
        'tranport_mean_bike'
        ]
    df = df.drop(columns=drop_columns)

    # Add year to column name
    for col in df.columns:
        if col != 'CT2020_GEOID':
            df.rename({col:'ACS'+str(year)+'_'+col}, axis='columns', inplace=True)
    return df

In [16]:
# Process and create new dataframe for ACS data
df_acs2017_processed = process_acs_data(df_acs2017, 2017)
df_acs2018_processed = process_acs_data(df_acs2018, 2018)
df_acs2019_processed = process_acs_data(df_acs2019, 2019)
df_acs2020_processed = process_acs_data(df_acs2020, 2020)
df_acs2021_processed = process_acs_data(df_acs2021, 2021)

In [27]:
# Merge data
dfs_to_merge = [df3, df_acs2017_processed, df_acs2018_processed, df_acs2019_processed, df_acs2020_processed, df_acs2021_processed]
df4 = reduce(lambda left, right: pd.merge(left, right, how='inner', on='CT2020_GEOID'), dfs_to_merge)
df4.shape

(21117, 64)

In [28]:
# Combine ACS data depending on the year
def combine_ACS_data(row, variable_name):
    '''
    This function takes in a default argument for the apply function.
    The function will combine 5 years of ACS data into 1 and show the data that corresponds to the crash year. 
    For example, if the crash happens in year 2021, ACS_ttl_pop will be the data from ACS2021_ttl_pop.
    '''
    if row['CRASH_YEAR'] == 2017:
        return row['ACS2017_'+variable_name]
    elif row['CRASH_YEAR'] == 2018:
        return row['ACS2018_'+variable_name]
    elif row['CRASH_YEAR'] == 2019:
        return row['ACS2019_'+variable_name]
    elif row['CRASH_YEAR'] == 2020:
        return row['ACS2020_'+variable_name]
    elif row['CRASH_YEAR'] == 2021:
        return row['ACS2021_'+variable_name]

df4['ACS_ttl_pop'] = df4.apply(combine_ACS_data, variable_name='ttl_pop', axis=1)
df4['ACS_pop_density_per_sq_mil'] = df4.apply(combine_ACS_data, variable_name='pop_density_per_sq_mil', axis=1)
df4['ACS_educ_less_hs_pct'] = df4.apply(combine_ACS_data, variable_name='educ_less_hs_pct', axis=1)
df4['ACS_educ_hs_pct'] = df4.apply(combine_ACS_data, variable_name='educ_less_hs_pct', axis=1)
df4['ACS_educ_bs_over_pct'] = df4.apply(combine_ACS_data, variable_name='educ_bs_over_pct', axis=1)
df4['ACS_median_household_inc'] = df4.apply(combine_ACS_data, variable_name='median_household_inc', axis=1)
df4['ACS_tranport_mean_car_pct'] = df4.apply(combine_ACS_data, variable_name='tranport_mean_car_pct', axis=1)
df4['ACS_tranport_mean_public_pct'] = df4.apply(combine_ACS_data, variable_name='tranport_mean_public_pct', axis=1)
df4['ACS_tranport_mean_bike_pct'] = df4.apply(combine_ACS_data, variable_name='tranport_mean_bike_pct', axis=1)
df4['ACS_avg_commmute_to_work_min'] = df4.apply(combine_ACS_data, variable_name='avg_commmute_to_work_min', axis=1)
df4['ACS_borough'] = df4.apply(combine_ACS_data, variable_name='borough', axis=1)

# Checking
df4[df4['CRASH_YEAR'].notnull()][['CRASH_YEAR','ACS_ttl_pop','ACS_pop_density_per_sq_mil','ACS_educ_less_hs_pct']]

,CRASH_YEAR,ACS_ttl_pop,ACS_pop_density_per_sq_mil,ACS_educ_less_hs_pct
0,2021,4630,72859.880,0.028763
1,2021,4630,72859.880,0.028763
2,2021,4630,72859.880,0.028763
3,2021,4630,72859.880,0.028763
4,2021,4630,72859.880,0.028763
...,...,...,...,...
21112,2017,2790,16419.860,0.080773
21113,2017,1153,20825.680,0.151272
21114,2017,2800,33348.360,0.263290
21115,2017,4665,60324.900,0.211258


In [29]:
df4.head()

,CRASH_DATE,ZIP_CODE,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,NUMBER_OF_CYCLIST_INJURED,NUMBER_OF_CYCLIST_KILLED,CT2020_GEOID,CRASH_YEAR-MONTH,CRASH_YEAR,ACS2017_ttl_pop,...,ACS_pop_density_per_sq_mil,ACS_educ_less_hs_pct,ACS_educ_hs_pct,ACS_educ_bs_over_pct,ACS_median_household_inc,ACS_tranport_mean_car_pct,ACS_tranport_mean_public_pct,ACS_tranport_mean_bike_pct,ACS_avg_commmute_to_work_min,ACS_borough
0,2021-12-14,11217.0,1.0,0.0,1,0,36047003300,2021-12,2021,3512,...,72859.88,0.028763,0.028763,0.786093,126101.0,0.062039,0.547954,0.041583,33.0,47
1,2021-09-08,11217.0,1.0,0.0,1,0,36047003300,2021-09,2021,3512,...,72859.88,0.028763,0.028763,0.786093,126101.0,0.062039,0.547954,0.041583,33.0,47
2,2021-09-20,11217.0,1.0,0.0,1,0,36047003300,2021-09,2021,3512,...,72859.88,0.028763,0.028763,0.786093,126101.0,0.062039,0.547954,0.041583,33.0,47
3,2021-09-20,11201.0,1.0,0.0,1,0,36047003300,2021-09,2021,3512,...,72859.88,0.028763,0.028763,0.786093,126101.0,0.062039,0.547954,0.041583,33.0,47
4,2021-05-24,11217.0,1.0,0.0,1,0,36047003300,2021-05,2021,3512,...,72859.88,0.028763,0.028763,0.786093,126101.0,0.062039,0.547954,0.041583,33.0,47


In [30]:
# Drop unneeded columns
drop_columns = [
    'ACS2017_ttl_pop', 
    'ACS2017_pop_density_per_sq_mil', 
    'ACS2017_educ_less_hs_pct', 
    'ACS2017_educ_hs_pct', 
    'ACS2017_educ_bs_over_pct', 
    'ACS2017_median_household_inc', 
    'ACS2017_tranport_mean_car_pct', 
    'ACS2017_tranport_mean_public_pct', 
    'ACS2017_tranport_mean_bike_pct', 
    'ACS2017_avg_commmute_to_work_min', 
    'ACS2017_borough', 
    'ACS2018_ttl_pop', 
    'ACS2018_pop_density_per_sq_mil', 
    'ACS2018_educ_less_hs_pct', 
    'ACS2018_educ_hs_pct', 
    'ACS2018_educ_bs_over_pct', 
    'ACS2018_median_household_inc', 
    'ACS2018_tranport_mean_car_pct', 
    'ACS2018_tranport_mean_public_pct', 
    'ACS2018_tranport_mean_bike_pct', 
    'ACS2018_avg_commmute_to_work_min', 
    'ACS2018_borough', 
    'ACS2019_ttl_pop', 
    'ACS2019_pop_density_per_sq_mil', 
    'ACS2019_educ_less_hs_pct', 
    'ACS2019_educ_hs_pct', 
    'ACS2019_educ_bs_over_pct', 
    'ACS2019_median_household_inc', 
    'ACS2019_tranport_mean_car_pct', 
    'ACS2019_tranport_mean_public_pct', 
    'ACS2019_tranport_mean_bike_pct', 
    'ACS2019_avg_commmute_to_work_min', 
    'ACS2019_borough', 
    'ACS2020_ttl_pop', 
    'ACS2020_pop_density_per_sq_mil', 
    'ACS2020_educ_less_hs_pct', 
    'ACS2020_educ_hs_pct', 
    'ACS2020_educ_bs_over_pct', 
    'ACS2020_median_household_inc', 
    'ACS2020_tranport_mean_car_pct', 
    'ACS2020_tranport_mean_public_pct', 
    'ACS2020_tranport_mean_bike_pct', 
    'ACS2020_avg_commmute_to_work_min', 
    'ACS2020_borough', 
    'ACS2021_ttl_pop', 
    'ACS2021_pop_density_per_sq_mil', 
    'ACS2021_educ_less_hs_pct', 
    'ACS2021_educ_hs_pct', 
    'ACS2021_educ_bs_over_pct', 
    'ACS2021_median_household_inc', 
    'ACS2021_tranport_mean_car_pct', 
    'ACS2021_tranport_mean_public_pct', 
    'ACS2021_tranport_mean_bike_pct', 
    'ACS2021_avg_commmute_to_work_min', 
    'ACS2021_borough']
df5 = df4.drop(columns=drop_columns)
df5.columns

Index(['CRASH_DATE', 'ZIP_CODE', 'NUMBER_OF_PERSONS_INJURED',
       'NUMBER_OF_PERSONS_KILLED', 'NUMBER_OF_CYCLIST_INJURED',
       'NUMBER_OF_CYCLIST_KILLED', 'CT2020_GEOID', 'CRASH_YEAR-MONTH',
       'CRASH_YEAR', 'ACS_ttl_pop', 'ACS_pop_density_per_sq_mil',
       'ACS_educ_less_hs_pct', 'ACS_educ_hs_pct', 'ACS_educ_bs_over_pct',
       'ACS_median_household_inc', 'ACS_tranport_mean_car_pct',
       'ACS_tranport_mean_public_pct', 'ACS_tranport_mean_bike_pct',
       'ACS_avg_commmute_to_work_min', 'ACS_borough'],
      dtype='object')

# Group by Census Tract and Crash Date

In [31]:
df6 = df5.copy()

In [162]:
# # Group by (date, census tract), then sum by death and injury number, use max for the ACS numbers

# # Method 1
# agg_func_math = {
#     'NUMBER_OF_PERSONS_INJURED': ['sum'],
#     'NUMBER_OF_PERSONS_KILLED': ['sum'],
#     'NUMBER_OF_CYCLIST_INJURED': ['sum'],
#     'NUMBER_OF_CYCLIST_KILLED': ['sum'],
#     'ACS_ttl_pop': ['median'],
#     'ACS_pop_density_per_sq_mil': ['median'],
#     'ACS_educ_less_hs_pct': ['median'],
#     'ACS_educ_hs_pct': ['median'],
#     'ACS_educ_bs_over_pct': ['median'],
#     'ACS_median_household_inc': ['median'],
#     'ACS_tranport_mean_car_pct': ['median'],
#     'ACS_tranport_mean_public_pct': ['median'],
#     'ACS_tranport_mean_bike_pct': ['median'],
#     'ACS_avg_commmute_to_work_min': ['median'],
#     'ACS_borough': ['median']
# }
# df7 = df6.groupby(['CRASH_DATE','CT2020_GEOID'], as_index=False).agg(agg_func_math)
# df7.columns = df7.columns.droplevel(-1)
# df7

,CRASH_DATE,CT2020_GEOID,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,NUMBER_OF_CYCLIST_INJURED,NUMBER_OF_CYCLIST_KILLED,ACS_ttl_pop,ACS_pop_density_per_sq_mil,ACS_educ_less_hs_pct,ACS_educ_hs_pct,ACS_educ_bs_over_pct,ACS_median_household_inc,ACS_tranport_mean_car_pct,ACS_tranport_mean_public_pct,ACS_tranport_mean_bike_pct,ACS_avg_commmute_to_work_min,ACS_borough
0,2017-01-01,36047003300,1.0,0.0,1,0,3512.0,55267.51,0.058802,0.058802,0.720691,131046.0,0.110871,0.692839,0.030630,31.0,47.0
1,2017-01-01,36047041700,1.0,0.0,1,0,3215.0,58995.98,0.376834,0.376834,0.138889,35749.0,0.135262,0.658143,0.000000,39.0,47.0
2,2017-01-01,36047115600,1.0,0.0,1,0,4091.0,66279.92,0.320384,0.320384,0.073517,18131.0,0.215111,0.676444,0.008000,47.0,47.0
3,2017-01-01,36061000202,1.0,0.0,1,0,7789.0,65067.35,0.296856,0.296856,0.300807,35444.0,0.091259,0.599153,0.056989,36.0,61.0
4,2017-01-01,36061013900,1.0,0.0,1,0,9600.0,136919.40,0.046196,0.046196,0.728733,112640.0,0.052184,0.474413,0.033011,29.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20878,2021-12-31,36061018300,1.0,0.0,1,0,8578.0,99039.85,0.063998,0.063998,0.764167,129909.0,0.068088,0.495654,0.046358,34.0,61.0
20879,2021-12-31,36081002800,1.0,0.0,1,0,3442.0,39613.49,0.125611,0.125611,0.262129,76628.0,0.484437,0.360498,0.022071,44.0,81.0
20880,2021-12-31,36081018402,1.0,0.0,1,0,2628.0,32023.45,0.132900,0.132900,0.187242,82794.0,0.563847,0.349917,0.000000,44.0,81.0
20881,2021-12-31,36081055300,1.0,0.0,1,0,3114.0,58859.07,0.120232,0.120232,0.372767,77679.0,0.246867,0.576441,0.016917,42.0,81.0


In [32]:
# Group by (year-month, census tract), then sum by death and injury number, use max for the ACS numbers

# Method 1
agg_func_math = {
    'NUMBER_OF_PERSONS_INJURED': ['sum'],
    'NUMBER_OF_PERSONS_KILLED': ['sum'],
    'NUMBER_OF_CYCLIST_INJURED': ['sum'],
    'NUMBER_OF_CYCLIST_KILLED': ['sum'],
    'ACS_ttl_pop': ['median'],
    'ACS_pop_density_per_sq_mil': ['median'],
    'ACS_educ_less_hs_pct': ['median'],
    'ACS_educ_hs_pct': ['median'],
    'ACS_educ_bs_over_pct': ['median'],
    'ACS_median_household_inc': ['median'],
    'ACS_tranport_mean_car_pct': ['median'],
    'ACS_tranport_mean_public_pct': ['median'],
    'ACS_tranport_mean_bike_pct': ['median'],
    'ACS_avg_commmute_to_work_min': ['median'],
    'ACS_borough': ['median']
}
df7 = df6.groupby(['CRASH_YEAR-MONTH','CT2020_GEOID'], as_index=False).agg(agg_func_math)
df7.columns = df7.columns.droplevel(-1)
df7

,CRASH_YEAR-MONTH,CT2020_GEOID,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,NUMBER_OF_CYCLIST_INJURED,NUMBER_OF_CYCLIST_KILLED,ACS_ttl_pop,ACS_pop_density_per_sq_mil,ACS_educ_less_hs_pct,ACS_educ_hs_pct,ACS_educ_bs_over_pct,ACS_median_household_inc,ACS_tranport_mean_car_pct,ACS_tranport_mean_public_pct,ACS_tranport_mean_bike_pct,ACS_avg_commmute_to_work_min,ACS_borough
0,2017-01,36005003900,1.0,0.0,1,0,6473.0,77540.32,0.473919,0.473919,0.089908,33552.0,0.094797,0.794726,0.000000,41.0,5.0
1,2017-01,36005005002,1.0,0.0,1,0,5583.0,126950.40,0.436416,0.436416,0.089884,30197.0,0.116468,0.727389,0.012372,49.0,5.0
2,2017-01,36005009800,1.0,0.0,1,0,5189.0,25972.46,0.153501,0.153501,0.277031,69927.0,0.508290,0.448848,0.004448,47.0,5.0
3,2017-01,36005013300,1.0,0.0,1,0,5934.0,95365.38,0.366209,0.366209,0.112064,24231.0,0.239107,0.655685,0.000000,42.0,5.0
4,2017-01,36005014100,1.0,0.0,1,0,5795.0,84717.05,0.274891,0.274891,0.176488,31446.0,0.204405,0.654185,0.000000,43.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17429,2021-12,36081146300,1.0,0.0,1,0,2951.0,23592.00,0.212372,0.212372,0.322942,74370.0,0.572161,0.269855,0.000000,45.0,81.0
17430,2021-12,36085002100,1.0,0.0,1,0,4854.0,14544.64,0.117307,0.117307,0.420475,61500.0,0.391740,0.483938,0.008344,47.0,85.0
17431,2021-12,36085003300,1.0,0.0,1,0,3758.0,11417.55,0.086939,0.086939,0.410877,99750.0,0.534097,0.314613,0.010315,45.0,85.0
17432,2021-12,36085013204,1.0,0.0,1,0,5246.0,12999.59,0.096117,0.096117,0.321124,97752.0,0.637581,0.324148,0.000000,43.0,85.0


In [33]:
# Create a numerical variable column to indicate the year of crash accident
df7['CRASH_YEAR'] = pd.to_datetime(df7['CRASH_YEAR-MONTH'].astype('datetime64[ns]')).dt.year

In [95]:
# # Create a numerical variable column to indicate the year of crash accident
# df7['CRASH_YEAR'] = pd.to_datetime(df7['CRASH_DATE']).dt.year

In [96]:
# # Create a numerical variable column to indicate the month of crash accident
# df7['CRASH_MONTH'] = pd.to_datetime(df7['CRASH_DATE']).dt.month

# Compute Ridability Score (Per Census Tract)

In [34]:
# Import length data
df_road_length = pd.read_excel("Data for ridability score.xlsx", sheet_name='road_by_ct_Statistics')
df_bikelane2021_class1 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2021_class1_Statistics')
df_bikelane2021_class2 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2021_class2_Statistics')
df_bikelane2021_class3 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2021_class3_Statistics')
df_bikelane2020_class1 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2020_class1_Statistics')
df_bikelane2020_class2 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2020_class2_Statistics')
df_bikelane2020_class3 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2020_class3_Statistics')
df_bikelane2019_class1 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2019_class1_Statistics')
df_bikelane2019_class2 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2019_class2_Statistics')
df_bikelane2019_class3 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2019_class3_Statistics')
df_bikelane2018_class1 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2018_class1_Statistics')
df_bikelane2018_class2 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2018_class2_Statistics')
df_bikelane2018_class3 = pd.read_excel("Data for ridability score.xlsx", sheet_name='bikelane2018_class3_Statistics')

In [35]:
df_road_length.head()

,OBJECTID,CT2020_GEOID,FREQUENCY,SUM_road_length
0,1,36005000100,204,25375.124023
1,2,36005000200,146,10533.120457
2,3,36005000400,157,14845.033377
3,4,36005001600,67,8179.186269
4,5,36005001901,99,6281.304454


## Clean, process, merge length data from ArcGIS output

In [36]:
# Drop columns
df_road_length = df_road_length.drop(columns=['OBJECTID', 'FREQUENCY'])

df_bikelane2021_class1 = df_bikelane2021_class1.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2021_class2 = df_bikelane2021_class2.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2021_class3 = df_bikelane2021_class3.drop(columns=['OBJECTID', 'FREQUENCY'])

df_bikelane2020_class1 = df_bikelane2020_class1.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2020_class2 = df_bikelane2020_class2.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2020_class3 = df_bikelane2020_class3.drop(columns=['OBJECTID', 'FREQUENCY'])

df_bikelane2019_class1 = df_bikelane2019_class1.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2019_class2 = df_bikelane2019_class2.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2019_class3 = df_bikelane2019_class3.drop(columns=['OBJECTID', 'FREQUENCY'])

df_bikelane2018_class1 = df_bikelane2018_class1.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2018_class2 = df_bikelane2018_class2.drop(columns=['OBJECTID', 'FREQUENCY'])
df_bikelane2018_class3 = df_bikelane2018_class3.drop(columns=['OBJECTID', 'FREQUENCY'])

In [41]:
df_bikelane2018_class3.head()

,CT2020_GEOID,class3_length
0,36005000400,543.805762
1,36005001600,802.179219
2,36005001901,33.758175
3,36005001903,614.580960
4,36005002500,153.850308


In [42]:
# Rename the length column (so that all have the same name)
df_road_length = df_road_length.rename({'SUM_road_length':'road_length'}, axis='columns')

df_bikelane2021_class1 = df_bikelane2021_class1.rename({'SUM_bikelane2021_class1_length':'class1_length'}, axis='columns')
df_bikelane2021_class2 = df_bikelane2021_class2.rename({'SUM_bikelane2021_class2_length':'class2_length'}, axis='columns')
df_bikelane2021_class3 = df_bikelane2021_class3.rename({'SUM_bikelane2021_class3_length':'class3_length'}, axis='columns')

df_bikelane2020_class1 = df_bikelane2020_class1.rename({'SUM_bikelane2020_class1_length':'class1_length'}, axis='columns')
df_bikelane2020_class2 = df_bikelane2020_class2.rename({'SUM_bikelane2020_class2_length':'class2_length'}, axis='columns')
df_bikelane2020_class3 = df_bikelane2020_class3.rename({'SUM_bikelane2020_class3_length':'class3_length'}, axis='columns')

df_bikelane2019_class1 = df_bikelane2019_class1.rename({'SUM_bikelane2019_class1_length':'class1_length'}, axis='columns')
df_bikelane2019_class2 = df_bikelane2019_class2.rename({'SUM_bikelane2019_class2_length':'class2_length'}, axis='columns')
df_bikelane2019_class3 = df_bikelane2019_class3.rename({'SUM_bikelane2019_class3_length':'class3_length'}, axis='columns')

df_bikelane2018_class1 = df_bikelane2018_class1.rename({'SUM_bikelane2018_class1_length':'class1_length'}, axis='columns')
df_bikelane2018_class2 = df_bikelane2018_class2.rename({'SUM_bikelane2018_class2_length':'class2_length'}, axis='columns')
df_bikelane2018_class3 = df_bikelane2018_class3.rename({'SUM_bikelane2018_class3_length':'class3_length'}, axis='columns')

In [43]:
# Convert from string to interger
df_bikelane2021_class1['CT2020_GEOID'] = df_bikelane2021_class1['CT2020_GEOID'].astype('Int64')
df_bikelane2021_class2['CT2020_GEOID'] = df_bikelane2021_class2['CT2020_GEOID'].astype('Int64')
df_bikelane2021_class3['CT2020_GEOID'] = df_bikelane2021_class3['CT2020_GEOID'].astype('Int64')

df_bikelane2020_class1['CT2020_GEOID'] = df_bikelane2020_class1['CT2020_GEOID'].astype('Int64')
df_bikelane2020_class2['CT2020_GEOID'] = df_bikelane2020_class2['CT2020_GEOID'].astype('Int64')
df_bikelane2020_class3['CT2020_GEOID'] = df_bikelane2020_class3['CT2020_GEOID'].astype('Int64')

df_bikelane2019_class1['CT2020_GEOID'] = df_bikelane2019_class1['CT2020_GEOID'].astype('Int64')
df_bikelane2019_class2['CT2020_GEOID'] = df_bikelane2019_class2['CT2020_GEOID'].astype('Int64')
df_bikelane2019_class3['CT2020_GEOID'] = df_bikelane2019_class3['CT2020_GEOID'].astype('Int64')

df_bikelane2018_class1['CT2020_GEOID'] = df_bikelane2018_class1['CT2020_GEOID'].astype('Int64')
df_bikelane2018_class2['CT2020_GEOID'] = df_bikelane2018_class2['CT2020_GEOID'].astype('Int64')
df_bikelane2018_class3['CT2020_GEOID'] = df_bikelane2018_class3['CT2020_GEOID'].astype('Int64')

In [44]:
df_bikelane2019_class3

,CT2020_GEOID,class3_length
0,36005000400,543.805762
1,36005001600,802.179219
2,36005001901,33.758175
3,36005001903,614.580881
4,36005002500,153.850308
...,...,...
799,36085013400,495.462004
800,36085014100,540.299882
801,36085014700,345.911377
802,36085016901,152.689016


In [45]:
# Make copies of the road_length column for later use (different years)
df_road_length2021 = df_road_length.copy()
df_road_length2020 = df_road_length.copy()
df_road_length2019 = df_road_length.copy()
df_road_length2018 = df_road_length.copy()

In [46]:
# Create a year column
df_road_length2021['CRASH_YEAR'] = 2021
df_road_length2020['CRASH_YEAR'] = 2020
df_road_length2019['CRASH_YEAR'] = 2019
df_road_length2018['CRASH_YEAR'] = 2018

df_bikelane2021_class1['CRASH_YEAR'] = 2021
df_bikelane2021_class2['CRASH_YEAR'] = 2021
df_bikelane2021_class3['CRASH_YEAR'] = 2021

df_bikelane2020_class1['CRASH_YEAR'] = 2020
df_bikelane2020_class2['CRASH_YEAR'] = 2020
df_bikelane2020_class3['CRASH_YEAR'] = 2020

df_bikelane2019_class1['CRASH_YEAR'] = 2019
df_bikelane2019_class2['CRASH_YEAR'] = 2019
df_bikelane2019_class3['CRASH_YEAR'] = 2019

df_bikelane2018_class1['CRASH_YEAR'] = 2018
df_bikelane2018_class2['CRASH_YEAR'] = 2018
df_bikelane2018_class3['CRASH_YEAR'] = 2018

In [47]:
df_bikelane2019_class3

,CT2020_GEOID,class3_length,CRASH_YEAR
0,36005000400,543.805762,2019
1,36005001600,802.179219,2019
2,36005001901,33.758175,2019
3,36005001903,614.580881,2019
4,36005002500,153.850308,2019
...,...,...,...
799,36085013400,495.462004,2019
800,36085014100,540.299882,2019
801,36085014700,345.911377,2019
802,36085016901,152.689016,2019


In [48]:
# For each year, merge length data of road and bike lane
dfs_length2021 = [df_road_length2021,df_bikelane2021_class1,df_bikelane2021_class2,df_bikelane2021_class3]
df_length2021_merge = reduce(lambda left, right: pd.merge(left, right, how='outer', on=['CT2020_GEOID','CRASH_YEAR']), dfs_length2021)

dfs_length2020 = [df_road_length2020,df_bikelane2020_class1,df_bikelane2020_class2,df_bikelane2020_class3]
df_length2020_merge = reduce(lambda left, right: pd.merge(left, right, how='outer', on=['CT2020_GEOID','CRASH_YEAR']), dfs_length2020)

dfs_length2019 = [df_road_length2019,df_bikelane2019_class1,df_bikelane2019_class2,df_bikelane2019_class3]
df_length2019_merge = reduce(lambda left, right: pd.merge(left, right, how='outer', on=['CT2020_GEOID','CRASH_YEAR']), dfs_length2019)

dfs_length2018 = [df_road_length2018,df_bikelane2018_class1,df_bikelane2018_class2,df_bikelane2018_class3]
df_length2018_merge = reduce(lambda left, right: pd.merge(left, right, how='outer', on=['CT2020_GEOID','CRASH_YEAR']), dfs_length2018)

In [49]:
df_length2018_merge.head()

,CT2020_GEOID,road_length,CRASH_YEAR,class1_length,class2_length,class3_length
0,36005000100,25375.124023,2018,NaN,NaN,NaN
1,36005000200,10533.120457,2018,367.482112,NaN,NaN
2,36005000400,14845.033377,2018,2750.435365,284.237382,543.805762
3,36005001600,8179.186269,2018,NaN,590.529576,802.179219
4,36005001901,6281.304454,2018,242.342993,124.153896,33.758175


In [50]:
# Concatenate the data from different years in the vertical direction
dfs_to_concat = [df_length2021_merge,
    df_length2020_merge,
    df_length2019_merge,
    df_length2018_merge
]
dfs_length_concat = pd.concat(dfs_to_concat, axis=0)
dfs_length_concat

,CT2020_GEOID,road_length,CRASH_YEAR,class1_length,class2_length,class3_length
0,36005000100,25375.124023,2021,NaN,NaN,NaN
1,36005000200,10533.120457,2021,367.552032,NaN,NaN
2,36005000400,14845.033377,2021,2757.315362,284.237382,543.805762
3,36005001600,8179.186269,2021,NaN,590.529593,802.179219
4,36005001901,6281.304454,2021,216.882735,124.153894,33.758175
...,...,...,...,...,...,...
2318,36085030301,19186.526249,2018,NaN,NaN,NaN
2319,36085030302,13414.056018,2018,NaN,NaN,NaN
2320,36085031901,6075.133246,2018,NaN,NaN,NaN
2321,36085031902,9082.195364,2018,NaN,NaN,NaN


## Merge bike lane & road length data with the crash & census data

In [51]:
df8 = df7.copy()

In [52]:
df_non2017 = df8[(df8['CRASH_YEAR']!=2017)]
df_non2017.shape

(14082, 18)

In [53]:
# Merge the road & bike lane length data to the crash data by CT2020_ID and year
df9 = pd.merge(df_non2017,dfs_length_concat, on=['CT2020_GEOID','CRASH_YEAR'], how ='left')
df9

,CRASH_YEAR-MONTH,CT2020_GEOID,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,NUMBER_OF_CYCLIST_INJURED,NUMBER_OF_CYCLIST_KILLED,ACS_ttl_pop,ACS_pop_density_per_sq_mil,ACS_educ_less_hs_pct,ACS_educ_hs_pct,...,ACS_tranport_mean_car_pct,ACS_tranport_mean_public_pct,ACS_tranport_mean_bike_pct,ACS_avg_commmute_to_work_min,ACS_borough,CRASH_YEAR,road_length,class1_length,class2_length,class3_length
0,2018-01,36005003100,1.0,0.0,1,0,1703.0,28511.08,0.288823,0.288823,...,0.217143,0.673143,0.000000,40.0,5.0,2018,3808.626585,NaN,NaN,562.016303
1,2018-01,36005008300,1.0,0.0,1,0,6519.0,78922.89,0.399219,0.399219,...,0.169690,0.744520,0.000000,48.0,5.0,2018,4977.191709,83.008465,587.162651,435.554757
2,2018-01,36005012901,1.0,0.0,1,0,4147.0,68682.77,0.328794,0.328794,...,0.178049,0.704878,0.000000,49.0,5.0,2018,3839.202371,NaN,507.820056,192.093230
3,2018-01,36005016700,1.0,0.0,1,0,3625.0,45612.34,0.254476,0.254476,...,0.282609,0.669187,0.000000,45.0,5.0,2018,4356.700925,NaN,397.296799,NaN
4,2018-01,36005017702,1.0,0.0,1,0,5786.0,106352.30,0.358862,0.358862,...,0.217875,0.634060,0.000000,42.0,5.0,2018,3533.739450,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14077,2021-12,36081146300,1.0,0.0,1,0,2951.0,23592.00,0.212372,0.212372,...,0.572161,0.269855,0.000000,45.0,81.0,2021,8651.943965,NaN,NaN,NaN
14078,2021-12,36085002100,1.0,0.0,1,0,4854.0,14544.64,0.117307,0.117307,...,0.391740,0.483938,0.008344,47.0,85.0,2021,17486.384575,NaN,3613.079342,2670.639969
14079,2021-12,36085003300,1.0,0.0,1,0,3758.0,11417.55,0.086939,0.086939,...,0.534097,0.314613,0.010315,45.0,85.0,2021,14472.643341,NaN,1743.854225,283.886351
14080,2021-12,36085013204,1.0,0.0,1,0,5246.0,12999.59,0.096117,0.096117,...,0.637581,0.324148,0.000000,43.0,85.0,2021,20526.895620,NaN,83.659631,640.448818


## Compute score

In [54]:
# Replace NaN by 0
df9['class1_length'] = df9['class1_length'].fillna(0)
df9['class2_length'] = df9['class2_length'].fillna(0)
df9['class3_length'] = df9['class3_length'].fillna(0)

In [55]:
# Compute the percent of bike lane and non-bike lane out of all roads
df9['class1_percent'] = df9['class1_length']/df9['road_length']
df9['class2_percent'] = df9['class2_length']/df9['road_length']
df9['class3_percent'] = df9['class3_length']/df9['road_length']
df9['no_bikelane_percent'] = (df9['road_length']-df9['class1_length']-df9['class2_length']-df9['class3_length'])/df9['road_length']

In [56]:
# Compute ridability score
df9['score1'] = df9['no_bikelane_percent']*(-1) + df9['class1_percent']*2 + df9['class2_percent']*1 + df9['class3_percent']*0
df9['score2'] = df9['no_bikelane_percent']*0 + df9['class1_percent']*3 + df9['class2_percent']*2 + df9['class3_percent']*1
df9['score3'] = df9['no_bikelane_percent']*1 + df9['class1_percent']*1000 + df9['class2_percent']*100 + df9['class3_percent']*10


In [57]:
df9

,CRASH_YEAR-MONTH,CT2020_GEOID,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,NUMBER_OF_CYCLIST_INJURED,NUMBER_OF_CYCLIST_KILLED,ACS_ttl_pop,ACS_pop_density_per_sq_mil,ACS_educ_less_hs_pct,ACS_educ_hs_pct,...,class1_length,class2_length,class3_length,class1_percent,class2_percent,class3_percent,no_bikelane_percent,score1,score2,score3
0,2018-01,36005003100,1.0,0.0,1,0,1703.0,28511.08,0.288823,0.288823,...,0.000000,0.000000,562.016303,0.000000,0.000000,0.147564,0.852436,-0.852436,0.147564,2.328076
1,2018-01,36005008300,1.0,0.0,1,0,6519.0,78922.89,0.399219,0.399219,...,83.008465,587.162651,435.554757,0.016678,0.117971,0.087510,0.777841,-0.626515,0.373485,30.127781
2,2018-01,36005012901,1.0,0.0,1,0,4147.0,68682.77,0.328794,0.328794,...,0.000000,507.820056,192.093230,0.000000,0.132272,0.050035,0.817693,-0.685421,0.314579,14.545268
3,2018-01,36005016700,1.0,0.0,1,0,3625.0,45612.34,0.254476,0.254476,...,0.000000,397.296799,0.000000,0.000000,0.091192,0.000000,0.908808,-0.817616,0.182384,10.028020
4,2018-01,36005017702,1.0,0.0,1,0,5786.0,106352.30,0.358862,0.358862,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14077,2021-12,36081146300,1.0,0.0,1,0,2951.0,23592.00,0.212372,0.212372,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-1.000000,0.000000,1.000000
14078,2021-12,36085002100,1.0,0.0,1,0,4854.0,14544.64,0.117307,0.117307,...,0.000000,3613.079342,2670.639969,0.000000,0.206622,0.152727,0.640651,-0.434028,0.565972,22.830162
14079,2021-12,36085003300,1.0,0.0,1,0,3758.0,11417.55,0.086939,0.086939,...,0.000000,1743.854225,283.886351,0.000000,0.120493,0.019615,0.859891,-0.739398,0.260602,13.105359
14080,2021-12,36085013204,1.0,0.0,1,0,5246.0,12999.59,0.096117,0.096117,...,0.000000,83.659631,640.448818,0.000000,0.004076,0.031200,0.964724,-0.960648,0.039352,1.684290


In [68]:
df9.columns

Index(['CRASH_YEAR-MONTH', 'CT2020_GEOID', 'NUMBER_OF_PERSONS_INJURED',
       'NUMBER_OF_PERSONS_KILLED', 'NUMBER_OF_CYCLIST_INJURED',
       'NUMBER_OF_CYCLIST_KILLED', 'ACS_ttl_pop', 'ACS_pop_density_per_sq_mil',
       'ACS_educ_less_hs_pct', 'ACS_educ_hs_pct', 'ACS_educ_bs_over_pct',
       'ACS_median_household_inc', 'ACS_tranport_mean_car_pct',
       'ACS_tranport_mean_public_pct', 'ACS_tranport_mean_bike_pct',
       'ACS_avg_commmute_to_work_min', 'ACS_borough', 'CRASH_YEAR',
       'road_length', 'class1_length', 'class2_length', 'class3_length',
       'class1_percent', 'class2_percent', 'class3_percent',
       'no_bikelane_percent', 'score1', 'score2', 'score3'],
      dtype='object')

# Data for Thesis

In [94]:
df10 = df9.copy()

In [95]:
# Drop columns
drop_columns = [
   'CRASH_YEAR-MONTH',
   'CT2020_GEOID',
   'NUMBER_OF_PERSONS_INJURED',
   'NUMBER_OF_PERSONS_KILLED',
   # 'NUMBER_OF_CYCLIST_INJURED',
   # 'NUMBER_OF_CYCLIST_KILLED',
   # 'ACS_ttl_pop',
   # 'ACS_pop_density_per_sq_mil',
   'ACS_educ_less_hs_pct',
   'ACS_educ_hs_pct',
   # 'ACS_educ_bs_over_pct',
   # 'ACS_median_household_inc',
   # 'ACS_tranport_mean_car_pct',
   # 'ACS_tranport_mean_public_pct',
   'ACS_tranport_mean_bike_pct',
   'ACS_avg_commmute_to_work_min',
   # 'ACS_borough',
   # 'CRASH_YEAR',
   'road_length',
   'class1_length',
   'class2_length',
   'class3_length',
   'class1_percent',
   'class2_percent',
   'class3_percent',
   'no_bikelane_percent',
   # 'score1',
   'score2',
   'score3'
]
df10 = df10.drop(columns=drop_columns)

In [96]:
# Rename columns
df10 = df10.rename({
    'score1':'ridability_score',
    'NUMBER_OF_CYCLIST_INJURED':'cyclist_injuries',
    'NUMBER_OF_CYCLIST_KILLED':'cyclist_death',
    'ACS_ttl_pop':'ttl_pop',
    'ACS_pop_density_per_sq_mil':'pop_density',
    'ACS_median_household_inc':'income',
    'ACS_educ_bs_over_pct':'educ',
    'ACS_tranport_mean_car_pct':'car',
    'ACS_tranport_mean_public_pct':'public_transportation',
    'ACS_borough':'borough',
    'CRASH_YEAR':'crash_year',
    }, axis='columns')

In [97]:
# Drop data entry that has missing values (which cannot be used in regression in STATA)
df10 = df10.dropna()

# Export data

In [198]:
# Export data as csv file
df9.to_csv('Final Data/final_data_w_ridability_score.csv',index=False)

In [98]:
# Export thesis data
df10.to_csv('Final Data/final_data_w_ridability_score_for_thesis.csv',index=False)